## 🔧 1. Environment Setup & Dependencies

In [ ]:
# Import required libraries
import time
import os

print("🔄 Starting imports...")
start_time = time.time()

# Set environment to use gcloud auth (remove empty GOOGLE_APPLICATION_CREDENTIALS)
if 'GOOGLE_APPLICATION_CREDENTIALS' in os.environ and os.environ['GOOGLE_APPLICATION_CREDENTIALS'] == '':
    del os.environ['GOOGLE_APPLICATION_CREDENTIALS']
os.environ['GCLOUD_PROJECT'] = 'junoplus-dev'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

print(f"⏱️ Basic libraries imported: {time.time() - start_time:.2f}s")

# BigQuery & Vertex AI (import directly without immediate auth)
print("🔄 Importing BigQuery & Vertex AI...")
start = time.time()

from google.cloud import bigquery
from google.cloud import aiplatform

print(f"⏱️ Google Cloud libraries imported: {time.time() - start:.2f}s")

# Deep Learning libraries
print("🔄 Importing TensorFlow (this may take a while)...")
start = time.time()
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical
print(f"⏱️ TensorFlow imported: {time.time() - start:.2f}s")

# Scikit-learn utilities
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow logging

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Set up project configuration
PROJECT_ID = 'junoplus-dev'
REGION = 'us-central1'
DATASET_ID = 'junoplus_analytics'

# Initialize BigQuery client (actual connection happens here)
print("🔄 Initializing BigQuery client...")
start = time.time()
client = bigquery.Client(project=PROJECT_ID, location=REGION)
print(f"⏱️ BigQuery client initialized: {time.time() - start:.2f}s")

print("🔄 Initializing Vertex AI...")
start = time.time()
aiplatform.init(project=PROJECT_ID, location=REGION)
print(f"⏱️ Vertex AI initialized: {time.time() - start:.2f}s")

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# GPU check
print("🔄 Detecting GPU...")
start = time.time()
gpu_available = len(tf.config.list_physical_devices('GPU')) > 0
print(f"⏱️ GPU detection completed: {time.time() - start:.2f}s")

print(f"\n✅ Environment setup complete!")
print(f"📊 Project: {PROJECT_ID}")
print(f"🌍 Region: {REGION}")
print(f"💾 Dataset: {DATASET_ID}")
print(f"🔢 TensorFlow version: {tf.__version__}")
print(f"🎮 GPU Available: {gpu_available}")
print(f"⏱️ Total setup time: {time.time() - start_time:.2f}s")

🔄 Starting imports...
⏱️ Basic libraries imported: 1.62s
🔄 Importing BigQuery & Vertex AI...
⏱️ Basic libraries imported: 1.62s
🔄 Importing BigQuery & Vertex AI...


## 📊 2. Data Loading & Feature Engineering

### Target Variables (from mostUsedSettings):
- **y_heat**: Heat Level (0-3, 4 classes)
- **y_mode**: TENS Mode (0-3, 4 classes)
- **y_tens**: TENS Level (0-10, 11 classes)

All targets will be **one-hot encoded** for neural network training.

In [ ]:
# Define medication potency mapping (from synthetic data generator)
MEDICATION_POTENCY = {
    'Advil': 1.15,
    'Midol': 1.0,
    'Naproxen': 1.3,
    'Paracetamol': 0.9,
    'Ibuprofen': 1.1,
    'Voltaren': 1.05,
    'Cycle Support Supplement': 0.5,
    'Vitamin D': 0.3,
    'Birth Control Pill': 0.4,
    'Lo Loestrin FE': 0.45,
    'Drospirenone-EE': 0.5,
}

# List of hormonal medications
HORMONAL_MEDICATIONS = [
    'Birth Control Pill',
    'Lo Loestrin FE',
    'Drospirenone-EE'
]

In [ ]:
# Load data from BigQuery with comprehensive feature engineering
# Same query as Approach 2 for consistency
query = """
WITH 
-- Extract device size from device name
device_info AS (
  SELECT 
    sessionId,
    CASE 
      WHEN LOWER(deviceName) LIKE '%grand%' THEN 'Grand'
      WHEN LOWER(deviceName) LIKE '%petit%' THEN 'Petit'
      ELSE 'Unknown'
    END AS device_size
  FROM `junoplus-dev.junoplus_analytics.ml_training_data`
),

-- Calculate user-level historical preferences
user_history AS (
  SELECT 
    userId,
    AVG(target_heat_level) AS user_avg_heat,
    AVG(target_tens_mode) AS user_avg_mode,
    AVG(target_tens_level) AS user_avg_tens,
    APPROX_TOP_COUNT(target_heat_level, 1)[OFFSET(0)].value AS user_mode_heat,
    APPROX_TOP_COUNT(target_tens_mode, 1)[OFFSET(0)].value AS user_mode_mode,
    APPROX_TOP_COUNT(target_tens_level, 1)[OFFSET(0)].value AS user_mode_tens
  FROM `junoplus-dev.junoplus_analytics.ml_training_data`
  WHERE target_heat_level IS NOT NULL
    AND target_tens_level IS NOT NULL
    AND target_tens_mode IS NOT NULL
  GROUP BY userId
),

-- Main data with all features
main_data AS (
  SELECT 
    t.sessionId,
    t.userId AS user_id,
    t.therapyStartTime,
    
    -- TARGET VARIABLES
    target_heat_level AS y_heat,
    target_tens_mode AS y_mode,
    target_tens_level AS y_tens,
    
    -- ADJUSTMENT DELTA FEATURES
    (final_heat_level - target_heat_level) AS delta_heat,
    (final_tens_level - target_tens_level) AS delta_tens,
    (final_tens_mode - target_tens_mode) AS delta_mode,
    
    -- CYCLE CONTEXT FEATURES
    COALESCE(cycle_day, 15) AS days_since_period_start,
    is_period_day AS is_near_period,
    cycle_phase_estimated,
    period_pain_level,
    flow_level,
    
    -- MEDICATION CONTEXT FEATURES
    has_pain_medication,
    medication_count,
    active_medication_count,
    recent_medication_usage,
    pain_medication_adherence,
    
    -- USER CONTEXT
    age,
    age_group,
    cycle_length,
    period_length,
    days_since_signup,
    user_experience_level,
    
    -- SESSION CONTEXT
    session_hour,
    day_of_week,
    time_of_day_category,
    therapyDuration,
    
    -- PAIN & EFFECTIVENESS
    input_pain_level,
    pain_level_before,
    pain_level_after,
    pain_reduction,
    pain_reduction_percentage,
    was_effective,
    
    -- DEVICE INFO
    d.device_size,
    most_used_battery_level,
    
    -- USER HISTORICAL PREFERENCES
    h.user_avg_heat,
    h.user_avg_mode,
    h.user_avg_tens,
    h.user_mode_heat,
    h.user_mode_mode,
    h.user_mode_tens,
    
    -- DATA SPLIT (user-stable split)
    CASE 
      WHEN MOD(FARM_FINGERPRINT(t.userId), 10) < 7 THEN 'TRAIN'
      WHEN MOD(FARM_FINGERPRINT(t.userId), 10) < 9 THEN 'EVAL'
      ELSE 'TEST'
    END AS data_split
    
  FROM `junoplus-dev.junoplus_analytics.ml_training_data` t
  LEFT JOIN device_info d ON t.sessionId = d.sessionId
  LEFT JOIN user_history h ON t.userId = h.userId
  
  WHERE target_heat_level IS NOT NULL
    AND target_tens_level IS NOT NULL
    AND target_tens_mode IS NOT NULL
    AND session_quality = 'high_quality'
    AND user_made_adjustments = TRUE
)

SELECT * FROM main_data
"""

print("🔄 Loading data from BigQuery...")
df = client.query(query).to_dataframe()

# Rename user_id back to userId for compatibility
df.rename(columns={'user_id': 'userId'}, inplace=True)

print(f"✅ Loaded {len(df):,} sessions from BigQuery")
print(f"\n📊 Data split distribution:")
print(df['data_split'].value_counts())
print(f"\n👥 Unique users: {df['userId'].nunique():,}")

🔄 Loading data from BigQuery...


NameError: name 'client' is not defined

In [ ]:
# Display data overview
print("\n📋 Dataset Overview:")
print(df.head())

print("\n📊 Target Variable Distributions:")
print("\nHeat Level (y_heat):")
print(df['y_heat'].value_counts().sort_index())

print("\nTENS Mode (y_mode):")
print(df['y_mode'].value_counts().sort_index())

print("\nTENS Level (y_tens):")
print(df['y_tens'].value_counts().sort_index())

## 🔧 3. Data Preprocessing & Feature Scaling

### Critical for Neural Networks:
1. **Feature Scaling**: Standardize all continuous features (mean=0, std=1)
2. **One-Hot Encoding**: Convert categorical features to binary columns
3. **Target Encoding**: Convert targets to one-hot encoded vectors
4. **Train/Eval/Test Split**: Maintain user-stable split

In [ ]:
# Handle missing values
print("🔧 Preprocessing features...")

# Fill missing numerical values with median
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numerical_cols = [col for col in numerical_cols if col not in 
                  ['y_heat', 'y_mode', 'y_tens', 'sessionId', 'userId']]

for col in numerical_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median(), inplace=True)

# Fill missing categorical values
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col not in 
                    ['sessionId', 'userId', 'data_split', 'therapyStartTime']]

for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna('Unknown', inplace=True)

print(f"✅ Missing values handled")

In [ ]:
# Encode categorical variables
print("🔢 Encoding categorical variables...")

encode_cols = ['device_size', 'time_of_day_category', 'cycle_phase_estimated', 
               'age_group', 'user_experience_level']

df_encoded = pd.get_dummies(df, columns=encode_cols, drop_first=True, dtype=int)

print(f"✅ Encoded {len(encode_cols)} categorical features")
print(f"📊 Total features after encoding: {len(df_encoded.columns)}")

In [ ]:
# Define feature columns
exclude_cols = ['sessionId', 'userId', 'therapyStartTime', 'data_split',
                'y_heat', 'y_mode', 'y_tens']

feature_cols = [col for col in df_encoded.columns if col not in exclude_cols]

print(f"\n✅ Feature columns: {len(feature_cols)} features")

In [ ]:
# Split data by user (user-stable split)
print("\n📊 Splitting data into TRAIN, EVAL, TEST sets...")

train_df = df_encoded[df_encoded['data_split'] == 'TRAIN'].copy()
eval_df = df_encoded[df_encoded['data_split'] == 'EVAL'].copy()
test_df = df_encoded[df_encoded['data_split'] == 'TEST'].copy()

print(f"\n✅ Data split complete:")
print(f"   TRAIN: {len(train_df):,} sessions")
print(f"   EVAL:  {len(eval_df):,} sessions")
print(f"   TEST:  {len(test_df):,} sessions")

In [ ]:
# Extract features and targets
print("\n🎯 Extracting features and targets...")

X_train = train_df[feature_cols].values
X_eval = eval_df[feature_cols].values
X_test = test_df[feature_cols].values

# Extract target variables (will be one-hot encoded later)
y_train_heat = train_df['y_heat'].values
y_train_mode = train_df['y_mode'].values
y_train_tens = train_df['y_tens'].values

y_eval_heat = eval_df['y_heat'].values
y_eval_mode = eval_df['y_mode'].values
y_eval_tens = eval_df['y_tens'].values

y_test_heat = test_df['y_heat'].values
y_test_mode = test_df['y_mode'].values
y_test_tens = test_df['y_tens'].values

print(f"✅ Features extracted: {X_train.shape[1]} features")

In [ ]:
# CRITICAL: Scale features for neural network
print("\n⚖️ Scaling features (StandardScaler)...")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_eval_scaled = scaler.transform(X_eval)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Features scaled successfully")
print(f"   Mean: {X_train_scaled.mean():.6f} (should be ≈ 0)")
print(f"   Std:  {X_train_scaled.std():.6f} (should be ≈ 1)")

In [ ]:
# One-hot encode target variables
print("\n🎯 One-hot encoding target variables...")

# Heat Level: 4 classes (0, 1, 2, 3)
y_train_heat_onehot = to_categorical(y_train_heat, num_classes=4)
y_eval_heat_onehot = to_categorical(y_eval_heat, num_classes=4)
y_test_heat_onehot = to_categorical(y_test_heat, num_classes=4)

# TENS Mode: 4 classes (0, 1, 2, 3)
y_train_mode_onehot = to_categorical(y_train_mode, num_classes=4)
y_eval_mode_onehot = to_categorical(y_eval_mode, num_classes=4)
y_test_mode_onehot = to_categorical(y_test_mode, num_classes=4)

# TENS Level: 11 classes (0, 1, 2, ..., 10)
y_train_tens_onehot = to_categorical(y_train_tens, num_classes=11)
y_eval_tens_onehot = to_categorical(y_eval_tens, num_classes=11)
y_test_tens_onehot = to_categorical(y_test_tens, num_classes=11)

print(f"✅ Targets one-hot encoded:")
print(f"   Heat: {y_train_heat_onehot.shape[1]} classes")
print(f"   Mode: {y_train_mode_onehot.shape[1]} classes")
print(f"   Tens: {y_train_tens_onehot.shape[1]} classes")

## 🏗️ 4. Build Multi-Output Neural Network

### Architecture:
```
Input Layer (n_features)
    ↓
Dense(256) + ReLU + Dropout(0.3)
    ↓
Dense(128) + ReLU + Dropout(0.3)
    ↓
Dense(64) + ReLU + Dropout(0.2)
    ↓
    ├─────────────┬─────────────┐
    ↓             ↓             ↓
Heat Head     Mode Head    Level Head
Dense(4)      Dense(4)     Dense(11)
Softmax       Softmax      Softmax
```

In [ ]:
# Build multi-output neural network
print("🏗️ Building multi-output neural network...")

# Input layer
input_layer = layers.Input(shape=(X_train_scaled.shape[1],), name='input')

# Shared base network (feature extraction)
x = layers.Dense(256, activation='relu', name='shared_dense_1')(input_layer)
x = layers.BatchNormalization(name='batch_norm_1')(x)
x = layers.Dropout(0.3, name='dropout_1')(x)

x = layers.Dense(128, activation='relu', name='shared_dense_2')(x)
x = layers.BatchNormalization(name='batch_norm_2')(x)
x = layers.Dropout(0.3, name='dropout_2')(x)

x = layers.Dense(64, activation='relu', name='shared_dense_3')(x)
x = layers.BatchNormalization(name='batch_norm_3')(x)
x = layers.Dropout(0.2, name='dropout_3')(x)

# Output heads (task-specific branches)

# Heat Level Head (4 classes)
heat_branch = layers.Dense(32, activation='relu', name='heat_dense')(x)
heat_output = layers.Dense(4, activation='softmax', name='heat_output')(heat_branch)

# TENS Mode Head (4 classes)
mode_branch = layers.Dense(32, activation='relu', name='mode_dense')(x)
mode_output = layers.Dense(4, activation='softmax', name='mode_output')(mode_branch)

# TENS Level Head (11 classes)
tens_branch = layers.Dense(32, activation='relu', name='tens_dense')(x)
tens_output = layers.Dense(11, activation='softmax', name='tens_output')(tens_branch)

# Create model
model = models.Model(
    inputs=input_layer,
    outputs=[heat_output, mode_output, tens_output],
    name='MultiOutput_DeviceSettings'
)

print("✅ Model architecture created!")
print(f"\n📊 Model Summary:")
model.summary()

## ⚙️ 5. Compile Model with Multi-Loss Configuration

### Loss Function:
- **Categorical Cross-Entropy** for each output head
- **Total Loss** = Heat Loss + Mode Loss + Level Loss

### Optimizer:
- **Adam** with learning rate = 0.001

In [ ]:
# Compile model with multiple losses
print("⚙️ Compiling model...")

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'heat_output': 'categorical_crossentropy',
        'mode_output': 'categorical_crossentropy',
        'tens_output': 'categorical_crossentropy'
    },
    loss_weights={
        'heat_output': 1.0,
        'mode_output': 1.0,
        'tens_output': 1.0
    },
    metrics={
        'heat_output': ['accuracy'],
        'mode_output': ['accuracy'],
        'tens_output': ['accuracy']
    }
)

print("✅ Model compiled successfully!")
print("\n📋 Configuration:")
print("   Optimizer: Adam (lr=0.001)")
print("   Loss: Categorical Cross-Entropy (3 heads)")
print("   Metrics: Accuracy (per head)")

## 🚀 6. Train the Multi-Output Model

### Training Configuration:
- **Epochs**: 50
- **Batch Size**: 64
- **Callbacks**: Early Stopping, Model Checkpoint, ReduceLROnPlateau

In [ ]:
# Set up callbacks
print("🔧 Setting up training callbacks...")

# Create models directory
os.makedirs('models/multioutput_approach', exist_ok=True)

# Early stopping to prevent overfitting
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Save best model
model_checkpoint = callbacks.ModelCheckpoint(
    'models/multioutput_approach/best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Reduce learning rate when stuck
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

print("✅ Callbacks configured:")
print("   • Early Stopping (patience=10)")
print("   • Model Checkpoint (save best)")
print("   • ReduceLROnPlateau (factor=0.5)")

In [ ]:
# Train the model
print("\n🚀 Training multi-output neural network...\n")

history = model.fit(
    X_train_scaled,
    {
        'heat_output': y_train_heat_onehot,
        'mode_output': y_train_mode_onehot,
        'tens_output': y_train_tens_onehot
    },
    validation_data=(
        X_eval_scaled,
        {
            'heat_output': y_eval_heat_onehot,
            'mode_output': y_eval_mode_onehot,
            'tens_output': y_eval_tens_onehot
        }
    ),
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    verbose=1
)

print("\n✅ Training complete!")

## 📊 7. Training History Visualization

Visualize loss and accuracy curves for all three output heads.

In [ ]:
# Plot training history
print("📊 Visualizing training history...\n")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Total Loss
axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0, 0].set_title('Total Loss', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Heat Level Accuracy
axes[0, 1].plot(history.history['heat_output_accuracy'], label='Train Acc', linewidth=2)
axes[0, 1].plot(history.history['val_heat_output_accuracy'], label='Val Acc', linewidth=2)
axes[0, 1].set_title('Heat Level Accuracy', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# TENS Mode Accuracy
axes[1, 0].plot(history.history['mode_output_accuracy'], label='Train Acc', linewidth=2)
axes[1, 0].plot(history.history['val_mode_output_accuracy'], label='Val Acc', linewidth=2)
axes[1, 0].set_title('TENS Mode Accuracy', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# TENS Level Accuracy
axes[1, 1].plot(history.history['tens_output_accuracy'], label='Train Acc', linewidth=2)
axes[1, 1].plot(history.history['val_tens_output_accuracy'], label='Val Acc', linewidth=2)
axes[1, 1].set_title('TENS Level Accuracy', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Training history visualized!")

## 🔮 8. Prediction & Evaluation on Test Set

Generate predictions for all three outputs and evaluate performance.

In [ ]:
# Make predictions on test set
print("🔮 Generating predictions on test set...")

# Get probability predictions for all three heads
predictions = model.predict(X_test_scaled, verbose=0)
y_pred_heat_proba, y_pred_mode_proba, y_pred_tens_proba = predictions

# Convert probabilities to class predictions using argmax
y_pred_heat = np.argmax(y_pred_heat_proba, axis=1)
y_pred_mode = np.argmax(y_pred_mode_proba, axis=1)
y_pred_tens = np.argmax(y_pred_tens_proba, axis=1)

print(f"✅ Predictions generated for {len(y_pred_heat):,} test samples")

In [ ]:
# Calculate performance metrics
print("\n📊 MULTI-OUTPUT DEEP LEARNING MODEL EVALUATION\n" + "="*80)

# Heat Level
heat_accuracy = accuracy_score(y_test_heat, y_pred_heat)
heat_f1 = f1_score(y_test_heat, y_pred_heat, average='weighted')

print("\n🔥 HEAT LEVEL PREDICTION:")
print(f"   Accuracy: {heat_accuracy:.4f}")
print(f"   F1 Score: {heat_f1:.4f}")
print("\n   Classification Report:")
print(classification_report(y_test_heat, y_pred_heat, 
                           target_names=['Heat 0', 'Heat 1', 'Heat 2', 'Heat 3']))

# TENS Mode
mode_accuracy = accuracy_score(y_test_mode, y_pred_mode)
mode_f1 = f1_score(y_test_mode, y_pred_mode, average='weighted')

print("\n⚡ TENS MODE PREDICTION:")
print(f"   Accuracy: {mode_accuracy:.4f}")
print(f"   F1 Score: {mode_f1:.4f}")
print("\n   Classification Report:")
print(classification_report(y_test_mode, y_pred_mode,
                           target_names=['Mode 0', 'Mode 1', 'Mode 2', 'Mode 3']))

# TENS Level
tens_accuracy = accuracy_score(y_test_tens, y_pred_tens)
tens_f1 = f1_score(y_test_tens, y_pred_tens, average='weighted')

print("\n🎯 TENS LEVEL PREDICTION:")
print(f"   Accuracy: {tens_accuracy:.4f}")
print(f"   F1 Score: {tens_f1:.4f}")
print("\n   Classification Report:")
print(classification_report(y_test_tens, y_pred_tens,
                           target_names=[f'Level {i}' for i in range(11)],
                           zero_division=0))

In [ ]:
# Visualize confusion matrices
print("\n📈 Generating confusion matrices...")

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Heat Level Confusion Matrix
cm_heat = confusion_matrix(y_test_heat, y_pred_heat)
sns.heatmap(cm_heat, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=[0, 1, 2, 3], yticklabels=[0, 1, 2, 3])
axes[0].set_title(f'Heat Level Confusion Matrix\nAccuracy: {heat_accuracy:.3f}')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# TENS Mode Confusion Matrix
cm_mode = confusion_matrix(y_test_mode, y_pred_mode)
sns.heatmap(cm_mode, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=[0, 1, 2, 3], yticklabels=[0, 1, 2, 3])
axes[1].set_title(f'TENS Mode Confusion Matrix\nAccuracy: {mode_accuracy:.3f}')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

# TENS Level Confusion Matrix (showing only levels 0-5 for visibility)
cm_tens = confusion_matrix(y_test_tens, y_pred_tens)
cm_tens_display = cm_tens[:6, :6]
sns.heatmap(cm_tens_display, annot=True, fmt='d', cmap='Oranges', ax=axes[2],
            xticklabels=range(6), yticklabels=range(6))
axes[2].set_title(f'TENS Level Confusion Matrix (0-5)\nAccuracy: {tens_accuracy:.3f}')
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('Actual')

plt.tight_layout()
plt.show()

print("✅ Confusion matrices visualized!")

## 📊 9. Final Summary & Comparison

Comprehensive summary of the multi-output deep learning approach.

In [ ]:
# Summary of results
print("\n" + "="*80)
print("🎯 MULTI-OUTPUT DEEP LEARNING APPROACH - FINAL SUMMARY")
print("="*80)

summary_df = pd.DataFrame({
    'Output Head': ['Heat Level', 'TENS Mode', 'TENS Level'],
    'Classes': [4, 4, 11],
    'Accuracy': [heat_accuracy, mode_accuracy, tens_accuracy],
    'F1 Score': [heat_f1, mode_f1, tens_f1]
})

print("\n")
print(summary_df.to_string(index=False))

# Calculate average metrics
avg_accuracy = summary_df['Accuracy'].mean()
avg_f1 = summary_df['F1 Score'].mean()

print("\n" + "-"*80)
print(f"📊 OVERALL PERFORMANCE:")
print(f"   Average Accuracy: {avg_accuracy:.4f}")
print(f"   Average F1 Score: {avg_f1:.4f}")

# Model statistics
total_params = model.count_params()
print(f"\n🔧 MODEL STATISTICS:")
print(f"   Total Parameters: {total_params:,}")
print(f"   Training Samples: {len(X_train):,}")
print(f"   Evaluation Samples: {len(X_eval):,}")
print(f"   Test Samples: {len(X_test):,}")

print("\n" + "="*80)
print("\n✅ Approach 3 (Multi-Output Deep Learning) Complete!")
print("\n📌 Key Advantages:")
print("   • Single unified model for all predictions")
print("   • Shared feature learning across tasks")
print("   • Parameter efficient architecture")
print("   • Captures inter-dependencies between settings")
print("   • Production-ready for deployment")
print("\n" + "="*80)

## 💾 10. Model Export & Deployment

Save the trained model and preprocessing artifacts for production deployment.

In [ ]:
# Save model and preprocessing artifacts
import joblib

print("💾 Saving model and artifacts...")

# Save the trained model
model.save('models/multioutput_approach/final_model.h5')
print("✅ Model saved: models/multioutput_approach/final_model.h5")

# Save the scaler
joblib.dump(scaler, 'models/multioutput_approach/feature_scaler.pkl')
print("✅ Scaler saved: models/multioutput_approach/feature_scaler.pkl")

# Save feature names
joblib.dump(feature_cols, 'models/multioutput_approach/feature_columns.pkl')
print("✅ Feature columns saved: models/multioutput_approach/feature_columns.pkl")

# Save training history
history_df = pd.DataFrame(history.history)
history_df.to_csv('models/multioutput_approach/training_history.csv', index=False)
print("✅ Training history saved: models/multioutput_approach/training_history.csv")

print("\n📦 All artifacts saved successfully!")
print("\n🚀 Ready for deployment to Vertex AI!")

## 🔍 11. Prediction Example (Optional)

Demonstrate how to use the trained model for inference.

In [ ]:
# Example: Make prediction for a single sample
print("🔍 Example Prediction:\n")

# Take first test sample
sample_idx = 0
sample_features = X_test_scaled[sample_idx:sample_idx+1]

# Make prediction
sample_pred = model.predict(sample_features, verbose=0)
pred_heat_proba, pred_mode_proba, pred_tens_proba = sample_pred

# Get class predictions
pred_heat = np.argmax(pred_heat_proba)
pred_mode = np.argmax(pred_mode_proba)
pred_tens = np.argmax(pred_tens_proba)

# Get actual values
actual_heat = y_test_heat[sample_idx]
actual_mode = y_test_mode[sample_idx]
actual_tens = y_test_tens[sample_idx]

print(f"Sample Index: {sample_idx}")
print("\n" + "-"*60)
print(f"{'Setting':<20} {'Predicted':<15} {'Actual':<15} {'Match'}")
print("-"*60)
print(f"{'Heat Level':<20} {pred_heat:<15} {actual_heat:<15} {'✅' if pred_heat == actual_heat else '❌'}")
print(f"{'TENS Mode':<20} {pred_mode:<15} {actual_mode:<15} {'✅' if pred_mode == actual_mode else '❌'}")
print(f"{'TENS Level':<20} {pred_tens:<15} {actual_tens:<15} {'✅' if pred_tens == actual_tens else '❌'}")
print("-"*60)

print("\n📊 Prediction Confidence:")
print(f"   Heat Level: {pred_heat_proba[0][pred_heat]:.2%}")
print(f"   TENS Mode:  {pred_mode_proba[0][pred_mode]:.2%}")
print(f"   TENS Level: {pred_tens_proba[0][pred_tens]:.2%}")